# Basic tutorial: qubit rotation [[Link]](https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html)



## Importing PennyLane and NumPy
---

In [ ]:
import pennylane as qml
from pennylane import numpy as np

## Creating a device
---

In [5]:
dev1 = qml.device("default.qubit", wires=1)
# Here, "wire" is the number of qubit registers

## Constructing the QNode
---

$$
R_x(\phi_1) = e^{-i\phi_1\sigma_x/2} = \begin{bmatrix} 
\cos\frac{\phi_1}{2} & -i\sin\frac{\phi_1}{2} \\
-i\sin\frac{\phi_1}{2} & \cos\frac{\phi_1}{2} \end{bmatrix} 
\\
R_y(\phi_2) = e^{-i\phi_2\sigma_y/2} = \begin{bmatrix} 
\cos\frac{\phi_2}{2} & -\sin\frac{\phi_2}{2} \\
\sin\frac{\phi_2}{2} & \cos\frac{\phi_2}{2} \end{bmatrix} 
$$

After these operations,
$$
|\psi\rangle = R_y(\phi_2)R_x(\phi_1)|0\rangle
$$

Finally, we measure the expectation value 
$$
\langle\psi|\sigma_z|\psi\rangle = \langle 0| R_x(\phi_1)^\dagger R_y(\phi_2)^\dagger \sigma_z R_y(\phi_2)R_x(\phi_1) |0 \rangle = \cos(\phi_1)\cos(\phi_2)
$$

In [3]:
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

Once we have written the quantum function, we convert it into a `QNode` running on device `dev1` by applying the `qnode()` decorator. directly above the function definition:

In [6]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [7]:
print(circuit([0.54, 0.12]))

0.8515405859048366


## Calculating quantum gradients
---

The gradient of the function `circuit` can be evaluated by utilizing the same quantum device (`dev1`) <br>
PennyLane incorporates both analytic differentiation, as well as numerical methods (such as the method of finite differences). <br>
Both of these are done automatically.<br>
<br>
We can differentiate by using the built-in `grad()` function. <br>
This returns another function, representing the gradient (i.e., the vector of partial derivatives) of `circuit`.

In [12]:
dcircuit = qml.grad(circuit, argnum=0)

`grad()` itself returns a function, representing the derivative of the QNode w.r.t. the argument specified in `argnum`.

In [13]:
print(dcircuit([0.54, 0.12]))

[array(-0.51043865), array(-0.1026782)]


### A note on arguments

Quantum circuit functions, being a restricted subset of Python functions, can also make use of multiple positional arguments and keyword arguments. <br>
Instead of one array argument, the quantum circuit function uses two positional arguments 

In [14]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

In this case, `argnum=0` will return the gradient w.r.t. only the first parameter `(phi1)` <br>
and `argnum=1` will give the gradient for `phi2`. <br>
To get the gradient w.r.t both parameters, we can use `argnum=[0,1]`:

In [15]:
dcircuit = qml.grad(circuit2, argnum=[0, 1])
print(dcircuit(0.54, 0.12))

(array(-0.51043865), array(-0.1026782))


## Optimization
---

**Definition** <br>

If using the default NumPy/Autograd interface, <br>
PennyLane provides a collection of optimizers based on gradient descent.

These optimizers accept a cost function and initial parameters, and utilize PennyLane's automatic differentiation to perform gradient descent.

See [Optimizers](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html) for details and docs of available optimizers

Next, let's make use of PennyLane's built-in optimizers to optimize the two circuit parameters $\phi_1$ and $\phi_2$ such that the qubit, originally in state $|0\rangle$, is rotated to be in state $|1\rangle$. <br>
This is equivalent to measuring a Pauli-Z expectation value of -1. since the $|1\rangle$ is an eigenvector of the Puali-Z matrix with eigenvalue $\lambda=-1$.

To do so, we need to define a **cost** function. By *minimizing* the cost function, the optimizer will determine the values of the circuit parameters that produce the desired outcome.

In this case, our desired outcome is a Pauli-Z expectation value of $-1$. Since we know that the Pauli-Z expectation is bound between $[-1,1]$, we can define our cost directly as the output of the QNode:

In [18]:
def cost(x):
    return circuit(x)

To begin our optimization, let's choose small initial values of $\phi_1$ and $\phi_2$:

In [19]:
init_params = np.array([0.011, 0.012])
print(cost(init_params))

0.9998675058299389


Finally, we use an optimizer to update the circuit parameters for 100 steps. We can use the built-in `GradientDescentOptimizer` class:

In [23]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i+1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.15266381e-18 3.14159265e+00]


The optimization converges after approximately 40 steps.

Substituting this into the theoretical result $\langle\psi|\sigma_z|\psi\rangle = \cos\phi_1\cos\phi_2$, <br>
we can verify that this is indeed one possible value of the circuit parameters that produce $\langle\psi|\sigma_z|\psi\rangle = -1$, <br>
resulting in the qubit being rotated to the state $|1\rangle$.

**Note** <br>
Some optimizers, such as `AdagradOptimizer`, have internal hyperparameters that are stored in the optimizer instance. <br>
These can be reset using the `reset()` method.

Continue on to the next tutorial, Gaussian transformation, to see a similar example using continuous-variable (CV) quantum nodes.